In [4]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

# Query by Allen Day, GooglCloud Developer Advocate (https://medium.com/@allenday)
query = """
#standardSQL
SELECT
  o.day,
  COUNT(DISTINCT(o.output_key)) AS recipients
FROM (
  SELECT
    TIMESTAMP_MILLIS((timestamp - MOD(timestamp,
          86400000))) AS day,
    output.output_pubkey_base58 AS output_key
  FROM
    `bigquery-public-data.bitcoin_blockchain.transactions`,
    UNNEST(outputs) AS output ) AS o
GROUP BY
  day
ORDER BY
  day
"""

query_job = client.query(query)

iterator = query_job.result(timeout=30)
rows = list(iterator)

# Transform the rows into a nice pandas dataframe
transactions = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))

# Look at the first 10 headlines
transactions.head(10)

,day,recipients
0,2009-01-09 00:00:00+00:00,0
1,2009-01-10 00:00:00+00:00,0
2,2009-01-11 00:00:00+00:00,0
3,2009-01-12 00:00:00+00:00,0
4,2009-01-13 00:00:00+00:00,0
5,2009-01-14 00:00:00+00:00,0
6,2009-01-15 00:00:00+00:00,0
7,2009-01-16 00:00:00+00:00,1
8,2009-01-17 00:00:00+00:00,0
9,2009-01-18 00:00:00+00:00,0
